In [6]:
import pygame as p

In [7]:
!git clone https://github.com/SkyXen/chess_images.git

fatal: destination path 'chess_images' already exists and is not an empty directory.


Main Engine of Chess

In [8]:
class GameState():
  
  def __init__(self):
    self.board = [
        ["bR","bN","bB","bQ","bK","bB","bN","bR"],
        ["bP","bP","bP","bP","bP","bP","bP","bP"],
        ["--","--","--","--","--","--","--","--"],
        ["--","--","--","--","--","--","--","--"],
        ["--","--","--","--","--","--","--","--"],
        ["--","--","--","--","--","--","--","--"],
        ["wP","wP","wP","wP","wP","wP","wP","wP"],
        ["wR","wN","wB","wQ","wK","wB","wN","wR"]]
    
    self.moveFunctions = {'P': self.getPawnMoves, 'R': self.getRookMoves, 'N': self.getKnightMoves,
                          'B': self.getBishopMoves, 'Q': self.getQueenMoves, 'K':self.getKingMoves}

    self.whiteToMove = True
    self.moveLog = []
    self.whiteKingLocation = (7,4)
    self.blackKingLocation = (0,4)
    self.checkMate = False
    self.staleMate = False

  def makeMove(self,move):
    self.board[move.startRow][move.startCol] = "--"
    self.board[move.endRow][move.endCol] = move.pieceMoved
    self.moveLog.append(move) #save the log of moved pieces
    self.whiteToMove = not self.whiteToMove #swap players
    #update King's location
    if move.pieceMoved == "wK":
      self.whiteKingLocation = (move.endRow, move.endCol)

    elif move.pieceMoved == "bK":
      self.blackKingLocation = (move.endRow, move.endCol)

    #pawn Promotion
    if move.isPawnPromotion:
      self.board[move.endRow][move.endCol] = move.pieceMoved[0] + 'Q'


  #undo the above moves
  def undoMove(self):
    if len(self.moveLog) != 0:
      move = self.moveLog.pop()
      self.board[move.startRow][move.startCol] = move.pieceMoved
      self.board[move.endRow][move.endCol] = move.pieceCaptured
      self.whiteToMove = not self.whiteToMove
      #update King's location
      if move.pieceMoved == "wK":
        self.whiteKingLocation = (move.startRow, move.startCol)

      elif move.pieceMoved == "bK":
        self.blackKingLocation = (move.startRow, move.startCol)

      self.checkMate = False
      self.staleMate = False

  def getValidMoves(self):
    moves = self.getAllPossibleMoves()
    for i in range(len(moves)-1,-1,-1):
      self.makeMove(moves[i])
      self.whiteToMove = not self.whiteToMove
      #checking if the king is in check or not
      if self.inCheck():
        moves.remove(moves[i])
      self.whiteToMove = not self.whiteToMove
      self.undoMove()

    if len(moves) == 0:
      if self.inCheck():
        self.checkMate = True
      else:
        self.staleMate = True
    else:
      self.checkMate = False
      self.staleMate = False

    return moves

  def inCheck(self):
    if self.whiteToMove:
      return self.sqUnderAttack(self.whiteKingLocation[0], self.whiteKingLocation[1])
    else:
      return self.sqUnderAttack(self.blackKingLocation[0], self.blackKingLocation[1])


  def sqUnderAttack(self,r,c):
    self.whiteToMove = not self.whiteToMove #switch to opp's turn
    oppMoves = self.getAllPossibleMoves()
    self.whiteToMove = not self.whiteToMove #switch back
    for move in oppMoves:
      if move.endRow == r and move.endCol == c:
        return True
    return False


  
  def getAllPossibleMoves(self):
    moves = []
    for r in range(len(self.board)):
      for c in range(len(self.board[r])):
        turn = self.board[r][c][0]
        if( turn == 'w' and self.whiteToMove) or (turn == 'b' and not self.whiteToMove):
          piece = self.board[r][c][1]
          self.moveFunctions[piece](r,c,moves)

    return moves

  #get all pawn moves at r,c and add to moves list
  def getPawnMoves(self,r,c,moves):
    if self.whiteToMove: #white pieces move
      #Moving pieces
      if c-1 >= 0:
        if self.board[r-1][c-1] == "--":
          moves.append(Move((r,c),(r-1,c-1),self.board))

      if c+1 <= 7:
        if self.board[r-1][c+1] == "--":
          moves.append(Move((r,c),(r-1,c+1),self.board))
    
      if self.board[r-1][c][0] == 'b':    #capturing forward piece
        moves.append(Move((r,c),(r-1,c),self.board))

    else: #black piece move
      #Moving pieces
      if r+1<=7:
        if c-1 >= 0:
          if self.board[r+1][c-1] == "--":
            moves.append(Move((r,c),(r+1,c-1),self.board))

        if c+1 <= 7:
          if self.board[r+1][c+1] == "--":
            moves.append(Move((r,c),(r+1,c+1),self.board))
       
        if c>=0 and c<=7:
          if self.board[r+1][c][0] == 'w':    #capturing forward piece
            moves.append(Move((r,c),(r+1,c),self.board))
      
      

  def getRookMoves(self,r,c,moves):
    directions = ((-1,0),(0,-1),(1,0),(0,1)) #up, left, down, right
    enemyColor = "b" if self.whiteToMove else "w"
    for d in directions:
      for i in range(1,8):
        endRow = r + d[0] * i
        endCol = c + d[1] * i

        if 0 <= endRow < 8 and 0<= endCol<8:
          endPiece = self.board[endRow][endCol]
          if endPiece == "--":
            moves.append(Move((r,c), (endRow,endCol), self.board))
          elif endPiece[0] == enemyColor:
            moves.append(Move((r,c), (endRow,endCol), self.board))
            break
          else:
            break
        else:
          break


  def getKnightMoves(self,r,c,moves):
    knightMoves = ((-2,-1),(-2,1),(-1,-2),(-1,2),(1,-2),(1,2),(2,-1),(2,1))
    allyColor = "w" if self.whiteToMove else "b"
    for m in knightMoves:
      endRow = r + m[0]
      endCol = c + m[1]

      if 0<= endRow <8 and 0<=endCol<8:
        endPiece = self.board[endRow][endCol]
        if endPiece[0] != allyColor:
          moves.append(Move((r,c),(endRow, endCol), self.board))

  def getBishopMoves(self,r,c,moves):
    directions = ((-1,-1),(-1,1),(1,-1),(1,1)) #diagonals
    enemyColor = "b" if self.whiteToMove else "w"
    for d in directions:
      for i in range(1,8):
        endRow = r + d[0] * i
        endCol = c + d[1] * i

        if 0 <= endRow < 8 and 0<= endCol<8:
          endPiece = self.board[endRow][endCol]
          if endPiece == "--":
            moves.append(Move((r,c), (endRow,endCol), self.board))
          elif endPiece[0] == enemyColor:
            moves.append(Move((r,c), (endRow,endCol), self.board))
            break
          else:
            break
        else:
          break

  def getKingMoves(self,r,c,moves):
    kingMoves = ((-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1))
    allyColor = "w" if self.whiteToMove else "b"

    for i in range(8):
      endRow = r + kingMoves[i][0]
      endCol = c + kingMoves[i][1]
      if 0<=endRow<8 and 0<=endCol<8:
        endPiece = self.board[endRow][endCol]
        if endPiece[0] != allyColor:
          moves.append(Move((r,c),(endRow,endCol),self.board))


  def getQueenMoves(self,r,c,moves):
    self.getRookMoves(r,c,moves)
    self.getBishopMoves(r,c,moves)

  


class Move():
  ranksToRows = {"1":7, "2":6, "3":5, "4":4,
                   "5":3, "6":2, "7":1, "8":0}
  rowsToRanks = {v: k for k, v in ranksToRows.items()}

  filesToCols = {"a":0, "b":1, "c":2, "d":3,
                   "e":4, "f":5, "g":6, "h":7}
  colsToFiles = {v: k for k, v in filesToCols.items()}
  
  def __init__(self, startSq, endSq, board):
    

    self.startRow = startSq[0]
    self.startCol = startSq[1]
    self.endRow = endSq[0]
    self.endCol = endSq[1]
    self.pieceMoved = board[self.startRow][self.startCol]
    self.pieceCaptured = board[self.endRow][self.endCol]
    self.isPawnPromotion = False
    if (self.pieceMoved == 'wP' and self.endRow == 0) or (self.pieceMoved == 'bP' and self.endRow == 7):
      self.isPawnPromotion = True
    self.moveID = self.startRow*1000 + self.startCol*100 + self.endRow *10 + self.endCol

  
  #overriding the equals method
  def __eq__(self, other):
    if isinstance(other,Move):
      return self.moveID == other.moveID
    return False

  def getChessNotation(self):
    #can be added to make this like real chess notation
    return self.getRankFile(self.startRow, self.startCol) + self.getRankFile(self.endRow, self.endCol)
     

  def getRankFile(self, r, c):
    return self.colsToFiles[c] + self.rowsToRanks[r]


Algorithm for making moves:

In [9]:
import random
 

pieceScore = { "K": 0, "Q": 10, "R":7, "B":3, "N":5, "P":1}
CHECKMATE = 1000
TWOCHECK = 700
STALEMATE = 0
DEPTH = 3

def findRandomMove(validMoves):
    return validMoves[random.randint(0,len(validMoves)-1)]

##Helper METHOD to make first recursive call##
def findBestMove(gs, validMoves):
    global nextMove
    nextMove = None
    random.shuffle(validMoves)
    moveNegaMaxAlphaBeta(gs, validMoves, DEPTH, -CHECKMATE, CHECKMATE, 1 if gs.whiteToMove else -1)
    return nextMove

def moveNegaMaxAlphaBeta(gs, validMoves, depth, alpha, beta, turnMultiplier):
    global nextMove
    if depth == 0:
        return turnMultiplier * scoreBoard(gs)
    
    maxScore = -CHECKMATE
    for move in validMoves:
        gs.makeMove(move)
        nextMoves = gs.getValidMoves()
        score = -moveNegaMaxAlphaBeta(gs,nextMoves,depth-1,-beta,-alpha,-turnMultiplier)
        if score>maxScore:
            maxScore = score
            if depth == DEPTH:
                nextMove=move
        gs.undoMove()
        if maxScore > alpha: #pruning
            alpha = maxScore
        if alpha >=beta:
            break
    return maxScore

##A positive score is good for white, a negative score is good for black

def scoreBoard(gs):
    if gs.inCheck():
        if gs.whiteToMove:
            return -TWOCHECK
        else:
            return TWOCHECK
    elif gs.checkMate:
        if gs.whiteToMove:
            return -CHECKMATE #black wins
        else:
            return CHECKMATE #white wins
    elif gs.staleMate:
        return STALEMATE
    score = 0
    for row in gs.board:
        for square in row:
            if square[0] == "w":
                score += pieceScore[square[1]]
            elif square[0] == "b":
                score -= pieceScore[square[1]]


    return score


Main Chess Game:

In [10]:
WIDTH = HEIGHT = 512
DIMENSION = 8 # 8*8 Chess board
SQ_SIZE = HEIGHT//DIMENSION
MAX_FPS = 15 # for animation
IMAGES = {}


def loadImages():
  pieces = ['wP', 'wR','wN','wB','wK','wQ','bP','bR','bN','bB','bK','bQ']
  for piece in pieces:
    IMAGES[piece] = p.transform.scale(p.image.load("chess_images/" + piece + ".png"), (SQ_SIZE, SQ_SIZE))

  #can be accessed using IMAGES['wP']


def main():
  p.init()
  screen = p.display.set_mode((WIDTH,HEIGHT))
  clock = p.time.Clock()
  screen.fill(p.Color("white"))
  gs = GameState()
  validMoves = gs.getValidMoves()
  moveMade = False
  loadImages()
  running = True
  sqSelected = () #atpresent no square selected, keep track of last clicked square (tuple:(row,col))
  playerClicks = [] #keep track of player clicks(two tuples: [(6,4),(4,4)])
  gameOver = False
  playerOne = False #If a human is playing white then this is true || if AI is playing black then this is true
  playerTwo = True #If AI is playing white then this is true || if human is playing black then this is true 
  numberOfChecksWhite = 3
  numberOfChecksBlack = 3

  while running:
    humanTurn  = (gs.whiteToMove and playerOne) or (not gs.whiteToMove and playerTwo)

    for e in p.event.get():
      if e.type == p.quit:
        running = False

      #mousehandler
      elif e.type == p.MOUSEBUTTONDOWN:
        if not gameOver and humanTurn:  
          location = p.mouse.get_pos() #(x,y) location of mouse
          col = location[0]//SQ_SIZE
          row = location[1]//SQ_SIZE
          if sqSelected == (row,col): #Already selected square
            sqSelected = () #Unselect it
            playerClicks = [] #clear it
          else:
            sqSelected = (row,col) #if not already selected then select it
            playerClicks.append(sqSelected)
          if len(playerClicks) == 2:
            move = Move(playerClicks[0],playerClicks[1],gs.board)
            
            print(move.getChessNotation(), numberOfChecksBlack, numberOfChecksWhite)
            for i in range(len(validMoves)):
              if move == validMoves[i]:
                gs.makeMove(validMoves[i])
                if gs.inCheck():
                  if gs.whiteToMove:
                    numberOfChecksWhite -= 1
                  else:
                    numberOfChecksBlack -= 1
                moveMade = True
                sqSelected = () #reset user clicks
                playerClicks = []
            if not moveMade:
              playerClicks = [sqSelected]
      
      #key_handler
      elif e.type == p.KEYDOWN:
        if e.key == p.K_z:
          gs.undoMove()
          moveMade = True
          gameOver = False

        if e.key == p.K_r:
          gs = GameState()
          validMoves = gs.getValidMoves()
          sqSelected = ()
          playerClicks = []
          moveMade = False
          gameOver = False
          numberOfChecksBlack = 3
          numberOfChecksWhite = 3

    ##AI MOVES##
    if not gameOver and not humanTurn:
      AIMove = findBestMove(gs,validMoves)
      if AIMove is None:
        AIMove = findRandomMove(validMoves)
      gs.makeMove(AIMove)
      if gs.inCheck():
        if gs.whiteToMove:
          numberOfChecksWhite -= 1
        else:
          numberOfChecksBlack -= 1
      print(AIMove.getChessNotation(), numberOfChecksBlack, numberOfChecksWhite)
            
      moveMade = True


    if moveMade:
      validMoves = gs.getValidMoves()
      moveMade = False
    
    drawGameState(screen, gs, validMoves, sqSelected)
    
    if gs.whiteToMove and gs.inCheck() and numberOfChecksWhite == 0:
      gameOver = True
      drawText(screen, "Black wins by ThreeChecks")

    else: 
      if gs.inCheck() and numberOfChecksBlack == 0:
        gameOver = True
        drawText(screen, "White wins by ThreeChecks")


    if gs.checkMate or gs.staleMate:
      gameOver = True
      drawText(screen, "STALEMATE" if gs.staleMate else "Black wins by CHECKMATE" if gs.whiteToMove else "White wins by CHECKMATE")

    clock.tick(MAX_FPS)
    p.display.flip()

  print(gs.board)



def drawGameState(screen,gs,validMoves,sqSelected):
  drawBoard(screen)#draw squares on board
  highlightSquares(screen,gs,validMoves,sqSelected)
  drawPieces(screen, gs.board) #draw pieces on board



def drawBoard(screen):
  colors = [p.Color("white"), p.Color("gray")]
  for r in range(DIMENSION):
    for c in range(DIMENSION):
      color = colors[((r + c) % 2)]
      p.draw.rect(screen, color, p.Rect(c*SQ_SIZE, r*SQ_SIZE, SQ_SIZE, SQ_SIZE))


def highlightSquares(screen, gs, validMoves, sqSelected):
  if sqSelected != ():
    r, c = sqSelected
    if gs.board[r][c][0] ==("w" if gs.whiteToMove else "b"):
      #highlight selected square
      s = p.Surface((SQ_SIZE,SQ_SIZE))
      s.set_alpha(100)
      s.fill(p.Color("blue"))
      screen.blit(s,(c*SQ_SIZE,r*SQ_SIZE))

      #highlight move around the selectd square
      s.fill(p.Color("green"))
      for move in validMoves:
        if move.startRow == r and move.startCol == c:
          screen.blit(s,(move.endCol*SQ_SIZE, move.endRow*SQ_SIZE))

      s.fill(p.Color("red"))
      if gs.inCheck() == True and gs.board[r][c][0] == "w":
        r,c = gs.whiteKingLocation
        screen.blit(s,(c*SQ_SIZE,r*SQ_SIZE))
      if gs.inCheck() == True and gs.board[r][c][0] == "b":
        r,c = gs.blackKingLocation
        screen.blit(s,(c*SQ_SIZE,r*SQ_SIZE))



#draw pieces on board using current game state
def drawPieces(screen,board):
  for r in range(DIMENSION):
    for c in range(DIMENSION):
      piece = board[r][c]
      if piece != "--":
        screen.blit(IMAGES[piece], p.Rect(c*SQ_SIZE, r*SQ_SIZE, SQ_SIZE, SQ_SIZE))

def drawText(screen, text):
  font = p.font.SysFont("Helvitca", 32,True,False)
  textObject = font.render(text,0,p.Color("Black"))
  textLocation = p.Rect(0,0,WIDTH,HEIGHT).move(WIDTH/2 - textObject.get_width()/2, HEIGHT/2 - textObject.get_height()/2)
  screen.blit(textObject, textLocation)

def drawEndGameText(screen,text):
  pass

if __name__ == "__main__":
  main()

a2b3 3 3
b8c6 3 3
g2f3 3 3
c7c6 3 3
c6d4 3 3
d2e3 3 3
d4f3 3 3
g1f3 3 2
c7d6 3 2
e1d2 3 2
d8g8 3 2
g8f6 3 2
f3g5 3 2
f6d8 3 2
d8a5 3 2
a1a5 3 1
f6e4 3 1


KeyboardInterrupt: 